## MIPs Adjustable Weight

In [1]:
from ortools.linear_solver import pywraplp
import pandas as pd
import random
import json

In [2]:
# def solve_student_project_assignment(students, projects, project_preferences, decay_factor=0.5, seat_fill_weight=1):
#     solver = pywraplp.Solver.CreateSolver('SCIP')
#     if not solver:
#         print("Solver not found")
#         return
#     # Decision variables for project assignment
#     x = {}
#     for s in students:
#         for p in projects:
#             if students[s]['major'] in projects[p]['majors']:
#                 x[s, p] = solver.BoolVar(f'x[{s},{p}]')

#     # Hard constraints
#     # Each student must be assigned 1 project
#     for s in students:
#         solver.Add(solver.Sum([x[s, p] for p in projects if (s, p) in x]) == 1)

#     # Do not exceed maximum number of students in a major for each project
#     for p in projects:
#         for m in projects[p]['majors']:
#             solver.Add(solver.Sum([x[s, p] for s in students if students[s]['major'] == m and (s, p) in x]) <= projects[p]['majors'][m])

#     # Soft constraints
#     # Exponential decay for project preferences
#     project_preferences_score = solver.Sum(
#         [(decay_factor ** project_preferences[s].index(p)) * x[s, p]
#          for s in project_preferences
#          for p in project_preferences[s]
#          if (s, p) in x])

#     # Score for filling seats in a project
#     seat_filling_score = solver.Sum([x[s, p] for s in students for p in projects if (s, p) in x]) * seat_fill_weight

#     # Objective function
#     solver.Maximize(project_preferences_score + seat_filling_score)

#     # Solve the problem
#     status = solver.Solve()

#     # Output the solution
#     solution = []
#     if status == pywraplp.Solver.OPTIMAL:
#         for s in students:
#             for p in projects:
#                 if (s, p) in x and x[s, p].solution_value() > 0.5:
#                     solution.append({'student': s, 'project': p})
#         return solution
#     else:
#         print('The problem does not have an optimal solution.')
#         return None

def solve_student_project_assignment(students, projects, decay_factor=0.5, seat_fill_weight=1, student_preference_weight=1):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("Solver not found")
        return

    # Decision variables for project assignment
    x = {}
    for student in students:
        for project in projects:
            if student['major'] in project['majors']:
                x[student['name'], project['name']] = solver.BoolVar(f'x[{student["name"]},{project["name"]}]')

    # Hard constraints
    # Each student must be assigned 1 project
    for student in students:
        solver.Add(solver.Sum([x[student['name'], project['name']] for project in projects if (student['name'], project['name']) in x]) == 1)

    # Do not exceed maximum number of students in a major for each project
    for project in projects:
        for m in project['majors']:
            solver.Add(solver.Sum([x[student['name'], project['name']] for student in students if student['major'] == m and (student['name'], project['name']) in x]) <= project['majors'][m])

    # Soft constraints
    # Exponential decay for project preferences
    project_preferences_score = solver.Sum(
        [(decay_factor ** student['project_preferences'].index(proj_name)) * x[student['name'], proj_name]
         for student in students
         for proj_name in student['project_preferences']
         if (student['name'], proj_name) in x])

    # Score for student preferences
    student_preferences_score = solver.Sum(
        [x[student['name'], project['name']] for student in students for project in projects for student_pref in student.get('student_preferences', []) if student_pref in [s['name'] for s in students] and project['name'] in [s['project_preferences'] for s in students if s['name'] == student_pref][0] and (student['name'], project['name']) in x and (student_pref, project['name']) in x]) * student_preference_weight

    # Score for filling seats in a project
    seat_filling_score = solver.Sum([x[student['name'], project['name']] for student in students for project in projects if (student['name'], project['name']) in x]) * seat_fill_weight

    # Objective function
    solver.Maximize(project_preferences_score + seat_filling_score + student_preferences_score)

    # Solve the problem
    status = solver.Solve()

    # Output the solution
    solution = []
    if status == pywraplp.Solver.OPTIMAL:
        for student in students:
            for project in projects:
                if (student['name'], project['name']) in x and x[student['name'], project['name']].solution_value() > 0.5:
                    solution.append({'student': student['name'], 'project': project['name']})
        return solution
    else:
        print('The problem does not have an optimal solution.')
        return None

In [3]:
students = [
    {'name': 'Alice', 'major': 'Computer Science', 'project_preferences': ['Project1', 'Project2'], 'student_preferences': ['Joe']},
    {'name': 'Bob', 'major': 'Engineering', 'project_preferences': ['Project2', 'Project1'], 'student_preferences': []},
    {'name': 'Joe', 'major': 'Engineering', 'project_preferences': ['Project2', 'Project1'], 'student_preferences': ['Ben']},
    {'name': 'Ben', 'major': 'Engineering', 'project_preferences': ['Project2', 'Project1'], 'student_preferences': []},
    {'name': 'Jared', 'major': 'Computer Science', 'project_preferences': ['Project1', 'Project2'], 'student_preferences': []},
    {'name': 'Bella', 'major': 'Computer Science', 'project_preferences': ['Project1', 'Project2'], 'student_preferences': []}
]

projects = [
    {'name': 'Project1', 'majors': {'Computer Science': 2, 'Engineering': 2}},
    {'name': 'Project2', 'majors': {'Computer Science': 2, 'Engineering': 2}}
]

solve_student_project_assignment(students, projects)

[{'student': 'Alice', 'project': 'Project1'},
 {'student': 'Bob', 'project': 'Project2'},
 {'student': 'Joe', 'project': 'Project1'},
 {'student': 'Ben', 'project': 'Project2'},
 {'student': 'Jared', 'project': 'Project1'},
 {'student': 'Bella', 'project': 'Project2'}]

## Import Data

In [4]:
# students_df = pd.read_csv('../data/students.csv', index_col=0)
# students_df = students_df.set_index('id')
# students_df = students_df.rename(columns={'Name': 'name',
#                                           'Degree': 'degree',
#                                           'Other_desc': 'desc',
#                                           'Major': 'major',
#                                           'project_order': 'project_preferences'})
# students_df['project_preferences'] = students_df['project_preferences'].apply(lambda x: x[1:-1].replace("'", "").replace(" ", "").split(','))
# students_df.head(3)

In [5]:
# students = []
# for index, row in students_df.iterrows():
#     student_dict = {
#         'name': row['name'],  # The student's name
#         'major': row['major'],  # The student's major
#         'project_preferences': row['project_preferences'],  # Assuming this is a list of project IDs
#         'student_preferences': []  # Assuming you do not have student preferences data; leave as empty list if that's the case
#     }
#     students.append(student_dict)

In [6]:
# projects_df = pd.read_csv('../data/projects.csv', index_col=0)
# projects_df = projects_df.rename(columns={'ID': 'id', 'Project name': 'project_name', 'Sponsor': 'sponsor', 'majors_requested': 'majors'})
# projects_df['majors'] = projects_df['majors'].apply(lambda x: json.loads(x))
# projects_df = projects_df.set_index('id')
# projects_df.head(3)

In [7]:
# projects = []
# for index, row in projects_df.iterrows():
#     project_dict = {
#         'name': row['project_name'],
#         'majors': row['majors']  # Assuming the majors column already contains a dictionary
#     }
#     projects.append(project_dict)

## Solve

In [8]:
# solution = solve_student_project_assignment(students, projects)
# solution = pd.DataFrame(solution)

In [9]:
# solution

In [10]:
# student_merge = pd.merge(left=students_df, right=solution, left_index=True, right_on='student')
# results = pd.merge(left=student_merge, right=projects_df, left_on='project', right_index=True).sort_values('student')

In [11]:
# results[['student', 'name', 'degree', 'major', 'desc', 'secondary_skills', 'project_preferences', 'project', 'project_name', 'sponsor', 'majors']].head(3)

In [12]:
# results.to_csv('../data/mips_solution.csv')